In [1]:
from __future__ import print_function
import os
import sys

In [2]:
# Someday this has to go in a config or kernel

#spark_home = '/opt/spark/latest'

#os.environ['SPARK_HOME'] = spark_home
#sys.path.insert(0, spark_home + "/python")
#sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.1-src.zip'))

#from pyspark.context import SparkContext
#from pyspark.context import SparkConf
#from pyspark.sql import SQLContext

In [3]:
#sc = SparkContext()
#sqlContext = SQLContext(sc)

In [1]:
df = sqlContext.read.load("/guoda/data/idigbio-scrap-20170605T152329.parquet")

In [4]:
print(df.count())
df.head(2)

250690


[Row(associatedsequences=None, barcodevalue=None, basisofrecord='preservedspecimen', bed=None, canonicalname='eucalyptus', catalognumber='rb01122693', class='magnoliopsida', collectioncode='esa', collectionid=None, collectionname=None, collector='t.b. flores', commonname=None, continent=None, coordinateuncertainty=None, country='brasil', countrycode=None, county='paraopeba', data=Row(coreid=None, dc:rights=None, dcterms:accessRights=None, dcterms:bibliographicCitation=None, dcterms:language=None, dcterms:license=None, dcterms:modified='2016-05-31 13:07:23.381267', dcterms:references=None, dcterms:rights=None, dcterms:rightsHolder='ESA', dcterms:source=None, dcterms:type='Collection', dwc:VerbatimEventDate=None, dwc:acceptedNameUsage=None, dwc:accessRights=None, dwc:associatedMedia='imagens3.jbrj.gov.br/fsi/server?type=image&source=rb/1/12/26/93/01122693.jpg', dwc:associatedOccurrences=None, dwc:associatedReferences=None, dwc:associatedSequences=None, dwc:associatedTaxa=None, dwc:basisO

In [3]:
df.printSchema()

root
 |-- associatedsequences: string (nullable = true)
 |-- barcodevalue: string (nullable = true)
 |-- basisofrecord: string (nullable = true)
 |-- bed: string (nullable = true)
 |-- canonicalname: string (nullable = true)
 |-- catalognumber: string (nullable = true)
 |-- class: string (nullable = true)
 |-- collectioncode: string (nullable = true)
 |-- collectionid: string (nullable = true)
 |-- collectionname: string (nullable = true)
 |-- collector: string (nullable = true)
 |-- commonname: string (nullable = true)
 |-- continent: string (nullable = true)
 |-- coordinateuncertainty: float (nullable = true)
 |-- country: string (nullable = true)
 |-- countrycode: string (nullable = true)
 |-- county: string (nullable = true)
 |-- data: struct (nullable = true)
 |    |-- coreid: string (nullable = true)
 |    |-- dc:rights: string (nullable = true)
 |    |-- dcterms:accessRights: string (nullable = true)
 |    |-- dcterms:bibliographicCitation: string (nullable = true)
 |    |-- dct

In [56]:
from pyspark.sql.types import *
def mk_field_set_from_df(df_schema):
    """Create a set of fieldname::type strings from a df schema"""
    field_set = set()
    prefix = ""
    for i in df_schema:
        #print(i)
        t = type(i.dataType)
        if t is not StringType and \
           t is not FloatType and \
           t is not IntegerType and \
           t is not TimestampType and \
           t is not BooleanType:
            #print(i.dataType)
            prefix = i.name
            
            # This only accommodates one level of nesting which is
            # consistant with our parquet-building technique. The
            # string manipulation on type makes it consistant with
            # the JSON returned by the API.
            for j in i.dataType:
                field_set.add(("{0}.{1}::{2}".format(prefix, j.name, 
                                            str(j.dataType)[:-4].lower())))

        else:
            field_set.add(("{0}::{1}".format(i.name, 
                                            str(i.dataType)[:-4].lower())))
    return field_set
df_field_set = mk_field_set_from_df(df.schema)
#print(df_field_set)

In [67]:
import requests

def mk_field_set_from_api():
    """Create a set of fieldname::type strings from the meta API endpoint"""
    meta_fields_records = (requests                                                                                                 
        .get("http://search.idigbio.org/v2/meta/fields/records")                                                 
        .json()                                                                                                  
    ) 
    field_set = set()                                                                                                              
    for k,v in meta_fields_records.items():
        if k == "data":                                                                                                             
            for kd,vd in v.items():                                                                                                 
                if vd.get("fieldName", False):                                                                                      
                    field_set.add("{0}::{1}".format(vd["fieldName"],
                                                        vd["type"]))
        elif v.get("fieldName", False):                                                                                               
            field_set.add("{0}::{1}".format(v["fieldName"], v["type"]))
        else:
            # non-data nested structures, only add top level key
            field_set.add("{0}::COMPLEX".format(k))
        
    return field_set
meta_field_set = mk_field_set_from_api()
#print(meta_field_set)

In [70]:
# The loaded df is a working data frame of course, what fields are in the
# meta endpoint that the df does not have? Are those fields currently 
# excluded in the parquet generation code or has Alex snuck in new
# structures lately?

print("\n".join(meta_field_set - df_field_set))

commonnames::string
recordids::string
datecollected::date
geopoint::geo_point
indexData::COMPLEX
datemodified::date
mediarecords::string
flags::string
